## Transfer Learning

คือการใช้โมเดลที่ถูกฝึกบน dataset ขนาดใหญ่ (เช่น ImageNet) แล้วนำความรู้มันมาช่วยงานใหม่ เช่น เอา ResNet ที่รู้จักรูปสัตว์ทั่วไป มาช่วยจำแนกสายพันธุ์แมว

### Transfer Learning สามารถทำได้ 2 แบบหลัก

#### Feature Extraction

- เราใช้เฉพาะ feature extractor ของโมเดล (เช่น convolutional layers) Freeze ทุก layer ไม่ให้เรียนรู้เพิ่ม เพิ่ม “classifier layer ใหม่” ตอนท้าย แล้วเทรนเฉพาะส่วนนั้น

- ตัวอย่างเช่น: ใช้ ResNet50 pretrained → แช่แข็งทุก layer → เทรนเฉพาะ fully connected layer สุดท้าย

#### Fine-Tuning (การปรับละเอียด)

- เรา “ไม่แช่แข็งทั้งหมด” แต่เปิดบางส่วนให้เรียนรู้เพิ่ม ใช้ learning rate ที่เล็ก เพื่อให้ weights เดิมค่อยๆ ปรับตาม domain ใหม่

- ตัวอย่างเช่น: ใช้ ResNet50 pretrained → unfreeze block สุดท้ายของ network → เทรนซ้ำเล็กน้อยให้เข้ากับ dataset ใหม่

#### สรุป
- Transfer Learning = เอาโมเดลเก่ามาใช้กับงานใหม่ (แนวคิดกว้าง)
- Fine-Tuning = วิธีหนึ่งในการทำ Transfer Learning (แนวทางเฉพาะ)

> “ทุก Fine-Tuning คือ Transfer Learning แต่ไม่ใช่ทุก Transfer Learning จะเป็น Fine-Tuning.”

### Install Libraries

- torch
- - PyTorch หลัก ใช้สร้างโมเดล Deep Learning, ทำคำนวณแบบ tensor (คล้าย NumPy แต่ optimized บน GPU), training neural network

- torchvision
- - เป็น extension สำหรับงาน Computer Vision เช่น โหลด datasets (MNIST, CIFAR, ImageNet), มี transforms (resize, normalize) และโมดูลโมเดลสำเร็จรูป (ResNet, VGG ฯลฯ)

- torchmetrics 
- - เอาไว้ใช้คำนวณ metrics สำหรับประเมินโมเดล เช่น Accuracy, Precision, Recall, F1-score โดยออกแบบมาให้ใช้งานง่ายและทำงานร่วมกับ PyTorch Lightning หรือ training loop ปกติ
  
- matplotlib
- - library สำหรับ การ plot กราฟและ visualization เช่น แสดงผลการ training loss, accuracy per epoch, ดูรูปภาพใน dataset ฯลฯ
  

- numpy
- - library สำหรับการคำนวณเชิงตัวเลข (Numerical Computation) ใช้สำหรับจัดการ array และ matrix (ข้อมูลหลายมิติ) ได้อย่างมีประสิทธิภาพ และรองรับการคำนวณทางคณิตศาสตร์ต่าง ๆ

### Set Up Environments

In [3]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

### Transfer Learning EfficientNet-B0

In [4]:
# Load a pre-trained model
model = models.efficientnet_b0(pretrained=True)

# Freeze all layers in the model
for param in model.parameters():
    param.requires_grad = False

/Users/morse/Documents/morse-workspaces/computer-vision-engineer/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/morse/Documents/morse-workspaces/computer-vision-engineer/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /Users/morse/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100.0%


In [5]:
# Print the modified model architecture
print(model)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [6]:
# Modify the final layer for a new classification task
num_features = model.classifier[1].in_features
num_classes = 2 # Example: binary classification
model.classifier[1] = nn.Linear(num_features, num_classes)

In [7]:
# print the modified model architecture
print(model)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

### Fine-Tuning EfficientNet-B0

In [8]:
# Load a pre-trained model
model = models.efficientnet_b0(pretrained=True)

# Fine-tune only the last few layers
for name, param in model.named_parameters():
    if "features.6" in name or "features.7" in name:  # block สุดท้าย ๆ
        param.requires_grad = True
    else:
        param.requires_grad = False

In [9]:
# Print the modified model architecture
print(model)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [10]:
num_features = model.classifier[1].in_features
num_classes = 2  # Example: binary classification
model.classifier[1] = nn.Linear(num_features, num_classes)

In [11]:
print(model)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat